<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%206%20-%20Aprendizaje%20por%20Transferencia%20y%20puesta%20a%20punto%20de%20parámetros%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Paso 1: Instalación de dependencias y configuración del entorno en GPU

In [1]:
#!pip install tensorflow-gpu==2.0.0.alpha0
%tensorflow_version 2.x

In [2]:
!pip install tqdm

### Descargar el dataset de Perros vs Gatos

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2021-09-22 19:44:23--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.146.128, 209.85.147.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.146.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   178MB/s    in 0.4s    

2021-09-22 19:44:24 (178 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Paso 2: Pre procesado del data set

### Importar las dependencias del proyecto

In [4]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.6.0'

### Descomprimir el dataset de Perros vs Gatos

In [5]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [6]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [7]:
zip_object.extractall("./")

In [8]:
zip_object.close()

### Configurar las rutas al dataset

In [9]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [10]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construir el Modelo

### Cargar un modelo pre entrenado (MobileNetV2)

In [11]:
IMG_SHAPE = (128, 128, 3)

In [12]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9420800/9406464 [==============================] - 0s 0us/step


In [13]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

### Congelar el modelo base

In [14]:
base_model.trainable = False

### Definir la cabecera personalizada para nuestra red neuronal

In [15]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [16]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [17]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [18]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Definir el modelo

In [19]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

### Compilar el modelo

In [21]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


### Crear generadores de datos

Redimensionar imágenes

    Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imágenes.
    
Por ejemplo: MobileNet (la arquitectura que nosotros usamos) soporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [22]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [23]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [24]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Entrenar el modelo 

In [25]:
model.fit_generator(train_generator, epochs=8, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/8
16/16 [==============================] - 44s 698ms/step - loss: 0.8722 - accuracy: 0.4275 - val_loss: 0.7839 - val_accuracy: 0.4990
Epoch 2/8
16/16 [==============================] - 9s 583ms/step - loss: 0.7444 - accuracy: 0.5455 - val_loss: 0.6832 - val_accuracy: 0.5940
Epoch 3/8
16/16 [==============================] - 9s 581ms/step - loss: 0.6498 - accuracy: 0.6405 - val_loss: 0.5987 - val_accuracy: 0.6860
Epoch 4/8
16/16 [==============================] - 9s 585ms/step - loss: 0.5711 - accuracy: 0.7130 - val_loss: 0.5273 - val_accuracy: 0.7490
Epoch 5/8
16/16 [==============================] - 9s 582ms/step - loss: 0.5037 - accuracy: 0.7770 - val_loss: 0.4677 - val_accuracy: 0.7830
Epoch 6/8
16/16 [==============================] - 9s 581ms/step - loss: 0.4480 - accuracy: 0.8130 - val_loss: 0.4166 - val_accuracy: 0.8210
Epoch 7/8
16/16 [==============================] - 9s 585ms/step - loss: 0.4005 - accuracy: 0.8390 - val_loss: 0.3727 - val_accuracy: 0.8520
Epoch 8/8
16

### Evaluar el modelo de aprendizaje por transferencia

In [26]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [27]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8769999742507935


## Puesta a punto de parámetros


Un par de cosas:

- NUNCA HAY QUE USAR la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente suficiente. En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.
- Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.

### Descongelar unas cuantas capas superiores del modelo

In [28]:
base_model.trainable = True

In [29]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [30]:
fine_tune_at = 100

In [31]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compilar el modelo para la puesta a punto

In [32]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


### Puesta a punto

In [33]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 18s 677ms/step - loss: 0.1961 - accuracy: 0.9150 - val_loss: 0.0914 - val_accuracy: 0.9670
Epoch 2/5
16/16 [==============================] - 10s 606ms/step - loss: 0.0282 - accuracy: 0.9975 - val_loss: 0.1038 - val_accuracy: 0.9660
Epoch 3/5
16/16 [==============================] - 10s 602ms/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.1074 - val_accuracy: 0.9670
Epoch 4/5
16/16 [==============================] - 10s 605ms/step - loss: 0.0061 - accuracy: 0.9995 - val_loss: 0.1123 - val_accuracy: 0.9650
Epoch 5/5
16/16 [==============================] - 10s 605ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.1143 - val_accuracy: 0.9690


### Evaluar el modelo re calibrado

In [34]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [35]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.968999981880188
